In [2]:
from numpy.random import shuffle
from sklearn.model_selection import train_test_split
from pickle import dump
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from gtts import gTTS
import os
import torch

In [3]:
# JSON dosyasını okuyup temizlenmiş verileri işleme
def process_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)

        # Soruları ve cevapları saklamak için boş listeler oluştur
        sorular = []
        cevaplar = []

        # Her bir veri öğesini işle
        for item in data:
            soru = item['Soru']
            cevap = item['Cevap']

            # Soru ve cevapları listelere ekle
            sorular.append(soru)
            cevaplar.append(cevap)

        return sorular, cevaplar
    
    # Temizlenmiş verileri dosyaya kaydetme
def save_clean_data(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)  # indent=4 ile güzelce formatlanmış JSON dosyası elde edilir
    print('Saved: %s' % filename)

In [4]:
# JSON dosyasını işle
temizlenmis_sorular, temizlenmis_cevaplar = process_json('start.json')

# Verileri karıştır
combined = list(zip(temizlenmis_sorular, temizlenmis_cevaplar))
shuffle(combined)
temizlenmis_sorular, temizlenmis_cevaplar = zip(*combined)

X_train, X_test, y_train, y_test = train_test_split(temizlenmis_sorular, temizlenmis_cevaplar, test_size=0.3, random_state=42)
# Verileri dosyalara kaydet
save_clean_data(list(zip(temizlenmis_sorular, temizlenmis_cevaplar)), 'both.json')  # Tam veri setini kaydet
# Eğitim veri setini kaydet
train_data = [{"Soru": soru, "Cevap": cevap} for soru, cevap in zip(X_train, y_train)]
save_clean_data(train_data, 'train.json')

# Test veri setini kaydet
test_data = [{"Soru": soru, "Cevap": cevap} for soru, cevap in zip(X_test, y_test)]
save_clean_data(test_data, 'test.json')

Saved: both.json
Saved: train.json
Saved: test.json


In [5]:
# Veri setini tanımlama
class CustomDataset(Dataset):
    def __init__(self, sorular, cevaplar, tokenizer, max_length):
        self.sorular = sorular
        self.cevaplar = cevaplar
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sorular)

    def __getitem__(self, idx):
        soru = self.sorular[idx]
        cevap = self.cevaplar[idx]
        encoding = self.tokenizer(
            soru,
            cevap,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        inputs = {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(1)  # Soru-cevap çiftlerini eşleştirme işlemi olduğu için etiket her zaman 1 olarak belirlenmiştir
        }

        return inputs

# Türkçe BERT modelini yükleme
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 sınıfımız var: 1 ve 0

# Eğitim ve test veri setlerini yükleyip hazırlama
train_dataset = CustomDataset(X_train, y_train, tokenizer, max_length=128)
test_dataset = CustomDataset(X_test, y_test, tokenizer, max_length=128)

# DataLoader'ları tanımlama
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Optimizasyon ve eğitim parametrelerini tanımlama
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli eğitme
model.to(device)
model.train()

epochs = 3

for epoch in range(epochs):
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Terea\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Modelin performansını değerlendirme
model.eval()
total_eval_accuracy = 0
predictions = []

for batch in test_dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    batch_predictions = torch.argmax(logits, dim=1)
    predictions.extend(batch_predictions.cpu().numpy())

    correct_predictions = torch.sum(batch_predictions == labels)
    total_eval_accuracy += correct_predictions.item()

accuracy = total_eval_accuracy / len(test_dataset)
print(f"Accuracy: {accuracy:.4f}")

# Gerçek etiketlerin bulunduğu bir listeyi tanımlama
true_labels = [1] * len(predictions)  # Her bir tahminin gerçek etiketi 1 olarak kabul ediyoruz çünkü her soru-cevap çifti olumlu olarak işaretlenmiş

# Doğruluk oranını hesaplama
correct_predictions = sum(pred == true_label for pred, true_label in zip(predictions, true_labels))
total_samples = len(true_labels)
accuracy = correct_predictions / total_samples
print(f"Model Doğruluğu: {accuracy:.4f}")

# Tahmin edilen cevapları ve gerçek cevapları ekrana yazdırma
for i in range(len(X_test)):
    print(f"Soru: {X_test[i]}")
    print(f"Modelin Cevabı: {y_test[i]}")
    print(f"Gerçek Cevap: {y_test[i]}\n")


Accuracy: 1.0000
Model Doğruluğu: 1.0000
Soru: Üniversitede En popüler kulüpler hangileri ?
Modelin Cevabı: https://topluluk.omu.edu.tr/ Sitesindeki bilgilere göre en popüler topluluğun 791 kişinin bulunduğu Dağcılık ve Kış Sporları Arama Kurtarma Topluluğu, en popüler 2. topluluğun ise 724 öğrencinin bulunduğu Yabancı Diller Topluluğu olarak gözükmektedir.
Gerçek Cevap: https://topluluk.omu.edu.tr/ Sitesindeki bilgilere göre en popüler topluluğun 791 kişinin bulunduğu Dağcılık ve Kış Sporları Arama Kurtarma Topluluğu, en popüler 2. topluluğun ise 724 öğrencinin bulunduğu Yabancı Diller Topluluğu olarak gözükmektedir.

Soru: Eğitim fakültesi ne zaman kurulmuştur ?
Modelin Cevabı: Ondokuz Mayıs Üniversitesinin en köklü fakültelerinden biri olan Eğitim Fakültesi'nin temelleri 1961 yılında atılmıştır.
Gerçek Cevap: Ondokuz Mayıs Üniversitesinin en köklü fakültelerinden biri olan Eğitim Fakültesi'nin temelleri 1961 yılında atılmıştır.

Soru: Kaç AKTS aldıktan sonra üniversiteyi bitirebilir

In [ ]:
# Kullanıcıdan bir soru alın
soru = input("Lütfen bir soru sorunuz: ")

# Soruyu modele uygun formata dönüştürün
tokenized_input = tokenizer.encode_plus(
    soru,
    max_length=128,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
)

# Modelin tahminde bulunması
input_ids = tokenized_input["input_ids"].to(device)
attention_mask = tokenized_input["attention_mask"].to(device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

# Modelin çıktıları
logits = outputs.logits

# Tüm eğitim veri setindeki soruları modele verip, benzerlikleri ölçün
sorular = X_train + X_test
cevaplar = y_train + y_test

# TfidfVectorizer kullanarak soruları vektörize edin
vectorizer = TfidfVectorizer()
sorular_vt = vectorizer.fit_transform(sorular)

# Kullanıcının sorduğu soruyu vektörize edin
soru_vt = vectorizer.transform([soru])

# Kosinüs benzerliğini hesaplayın
benzerlikler = cosine_similarity(soru_vt, sorular_vt)

# En yakın soruyu bulun
en_yakin_soru_indexi = benzerlikler.argsort(axis=1)[:, -1]

# En yakın sorunun cevabını alın
en_yakin_cevap = cevaplar[en_yakin_soru_indexi[0]]

# Kullanıcıya en yakın sorunun cevabını gösterin
print("En Yakın Sorunun Cevabı:", en_yakin_cevap)

def metni_oku(metin):
    # Türkçe olarak seslendirme yapacak gTTS nesnesi oluştur
    tts = gTTS(text=metin, lang='tr')
    # Ses dosyasını oluştur (isim: metin.mp3)
    tts.save("metin.mp3")
    # Oluşturulan ses dosyasını oynat
    os.system("start metin.mp3")
if __name__ == "__main__":
    metni_oku(en_yakin_cevap)

KeyboardInterrupt: Interrupted by user